In [1]:
%pylab notebook
import datacube
from datetime import datetime, timedelta
import pandas as pd
from fc.fractional_cover import fractional_cover
import pyproj
import csv

dc = datacube.Datacube(app='nbart-fieldsites')

Populating the interactive namespace from numpy and matplotlib


In [2]:
fieldsites = pd.read_csv(
    'jrsrp_20170430.csv',
    usecols=[
        'obs_key',
        "longitude",
        "latitude",
        "crust",
        "dist",
        "rock",
        "green",
        "crypto",
        "dead",
        "litter",
        "mid_g",
        "mid_d",
        "mid_b",
        "crn",
        "over_g",
        "over_d",
        "over_b",
        "persist",
        "num_points",
        "unoccluded",
        "image"
    ])

In [7]:
def query_cube(obs_key, lon, lat, image):
    sensor = 'ls'+image[1:2]
    path = image[8:11]
    row = image[12:15]
    date = image[16:20]+'-'+image[20:22]+'-'+image[22:24]
    target_day = datetime.strptime(date, '%Y-%m-%d')
    day_before = (target_day - timedelta(days=1)).strftime('%Y-%m-%d')
    day_after = (target_day + timedelta(days=1)).strftime('%Y-%m-%d')
    
    query = {'time': (day_before, day_after)}
    
    wgs_84 = pyproj.Proj(init='epsg:4326')
    aus_albers = pyproj.Proj(init='epsg:3577')

    x, y = pyproj.transform(wgs_84, aus_albers, lon, lat)
    
    y_max = y-30
    y_min = y+30
    x_max = x-30
    x_min = x+30
    query['x'] = (x_min, x_max)
    query['y'] = (y_max, y_min)
    query['crs'] = 'EPSG:3577'
        
    query['source_filter'] = dict(product=sensor+'_nbar_scene', sat_path=path, sat_row=row)

    data = dc.load(product=sensor+'_fc_albers', **query)

    if(data.coords):
        data = data.squeeze()
        point = data.sel(x=x,y=y, method='nearest')
        new_x = point.coords['x'].values
        new_y = point.coords['y'].values
        
        new_y_max = new_y-30
        new_y_min = new_y+30
        new_x_max = new_x+30
        new_x_min = new_x-30
        query_x = (new_x_min, new_x_max)
        query_y = (new_y_min,new_y_max)
        target_cells = data.sel(x=slice(*query_x),y=slice(*query_y))
        
        min_lon, min_lat = pyproj.transform(aus_albers, wgs_84, new_x_min, new_y_min)
        max_lon, max_lat = pyproj.transform(aus_albers, wgs_84, new_x_max, new_y_max)

        target_cells = target_cells.squeeze()
        
        BS_mean = float(target_cells.BS.mean())
        BS_std = float(target_cells.BS.std())
        PV_mean = float(target_cells.PV.mean())
        PV_std = float(target_cells.PV.std())
        NPV_mean = float(target_cells.NPV.mean())
        NPV_std = float(target_cells.NPV.std())
        UE_mean = float(target_cells.UE.mean())
        UE_std = float(target_cells.UE.std())
        
        x_size = target_cells.coords['x'].to_index().size
        y_size = target_cells.coords['y'].to_index().size
        
        return BS_mean, BS_std, PV_mean, PV_std, NPV_mean, NPV_std, UE_mean, UE_std, x_size * y_size
    else:
        return 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 0

In [8]:
with open('nbar_fieldsites_with_old_endmembers.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    columns = ['obs_key',
        'longitude',
        'latitude',
        'image',
        "crust",
        "dist",
        "rock",
        "green",
        "crypto",
        "dead",
        "litter",
        "mid_g",
        "mid_d",
        "mid_b",
        "crn",
        "over_g",
        "over_d",
        "over_b",
        "persist",
        "num_points",
        "unoccluded",
        "BS_mean",
        "BS_std",
        "PV_mean",
        "PV_std",
        "NPV_mean",
        "NPV_std",
        "UE_mean",
        "UE_std",
        'count'
        ]
    writer.writerow(columns)
    fieldsite_tuples = [tuple(x) for x in fieldsites.values]
    for fieldsite in fieldsite_tuples: 
        partialrow=(fieldsite[0],
                    fieldsite[1],
                    fieldsite[2],
                    fieldsite[3],
                    fieldsite[4],
                    fieldsite[5],
                    fieldsite[6],
                    fieldsite[7],
                    fieldsite[8],
                    fieldsite[9],
                    fieldsite[10],
                    fieldsite[11],
                    fieldsite[12],
                    fieldsite[13],
                    fieldsite[14],
                    fieldsite[15],
                    fieldsite[16],
                    fieldsite[17],
                    fieldsite[18],
                    fieldsite[19],
                    fieldsite[20])
        writer.writerow(partialrow+ query_cube(fieldsite[0],
                    fieldsite[1],
                    fieldsite[2],
                    fieldsite[20]))
        